## Imports

In [2]:
import tensorflow as tf
import numpy as np

## Load MNIST

In [7]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./tmp/data", one_hot=True)

Extracting ./tmp/data\train-images-idx3-ubyte.gz
Extracting ./tmp/data\train-labels-idx1-ubyte.gz
Extracting ./tmp/data\t10k-images-idx3-ubyte.gz
Extracting ./tmp/data\t10k-labels-idx1-ubyte.gz


In [8]:
print(len(mnist.train.images[0]))
print(len(mnist.train.labels[0]))

784
10


## 01 Placeholder

In [10]:
X = tf.placeholder(tf.float32, [None, 784], name="X")
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

## 02 Prediction

**ReLU Conv1**

In [16]:
W1 = tf.Variable(tf.random_normal(shape=[9, 9, 1, 256], stddev=0.01)) # filter_height, fliter_width, in_channel, out_channel(= # of filters)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="VALID") # strides=[1, stride, stride, 1]
L1 = tf.nn.relu(L1)
print(L1)

Tensor("Relu_3:0", shape=(?, 20, 20, 256), dtype=float32)


**PrimaryCaps**

In [21]:
W2 = tf.Variable(tf.random_normal(shape=[9, 9, 256, 256], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 2, 2, 1], padding="VALID")
L2 = tf.nn.relu(L2)
print(L2)
L2 = tf.reshape(L2, [-1, 6 * 6 * 32, 8])
print(L2)

Tensor("Relu_8:0", shape=(?, 6, 6, 256), dtype=float32)
Tensor("Reshape_5:0", shape=(?, 1152, 8), dtype=float32)


**squashing function**

In [31]:
def squash(s):
    epsilon=1e-7
    
    squared_norm = tf.sqrt(tf.reduce_sum(tf.square(s), keep_dims=True) + epsilon)
    squash_factor = squared_norm / (1. + squared_norm)
    unit_vector = s / squared_norm
    
    return squash_factor * unit_vector

In [33]:
L2 = squash(L2)
print(L2)

Tensor("mul_1:0", shape=(?, 1152, 8), dtype=float32)


## Cost function and Optimizer

In [ ]:
alpha = 0.0005

# cost = tf.add(margin_loss, alpha * reconstruction_loss)
# optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

## Accuracy

In [ ]:
is_correct = tf.equal(tf.argmax(predictions , axis=1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

## Training

In [ ]:
epochs = 15
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += c / total_batch

    print("Epoch:", "%04d" % (epoch + 1), "cost =", "{:.9f}".format(avg_cost))

print("Learning finished!")